In [8]:
# Example of predicting on new data
new_data = pd.read_csv('clearskypred.csv')  # Replace 'new1data.csv' with your new data file
new_features = new_data[['clearsky']]  # Example: selecting relevant features

# Normalize the new data using the same scaler
scaled_new_features = scaler.transform(new_features)

# Create sequences for the new data
def create_sequences(data, seq_length):
    X = []
    for i in range(len(data) - seq_length + 1):
        sequence = data[i:i + seq_length]
        X.append(sequence)
    return np.array(X)

# Example usage:
seq_length = 10  # Adjust this as per your needs
X_new = create_sequences(scaled_new_features, seq_length)

# Make predictions on new data
new_predictions = model.predict(X_new)

# Inverse transform predictions to original scale
new_predictions = scaler.inverse_transform(new_predictions)
new_predictions = new_predictions.squeeze()
# Print or use new_predictions as needed
print(f'the next 60 days  value generated {new_predictions[:60]}')
new_predictions = np.array(new_predictions)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
the next 60 days  value generated [6.9796743 6.9877954 6.9125123 6.7945027 6.706309  6.737358  6.820261
 6.9158163 6.9968514 7.0227985 7.01037   6.991084  6.961612  6.9035363
 6.8817782 6.858996  6.8673267 6.901821  6.951216  6.994352  7.0144296
 6.944822  6.8360333 6.793413  6.7353506 6.704031  6.6767945 6.6896567
 6.7685657 6.867928  6.9687843 7.0679884 7.1596065 7.209269  7.2481437
 7.2679663 7.2869883 7.293232  7.3172917 7.2864046 7.2367373 7.159658
 7.036342  6.95189   6.906763  6.8968287 6.895984  6.8872733 6.8909707
 6.9007726 6.939664  6.9581275 6.9780755 6.991504  6.992429  6.9438076
 6.9086814 6.895879  6.8876534 6.919152 ]


In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load your dataset
data = pd.read_csv('bangaluru data.csv')  # Replace with your dataset

# Select relevant features and target variable
features = data[['clearsky']]  # Example features
target = data['temp']  # Example target

# Normalize the data
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Create sequences for LSTM
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(data[i + seq_length])
    return np.array(sequences), np.array(targets)

seq_length = 10
X, y = create_sequences(scaled_features, seq_length)

# Split into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, X.shape[2])))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2, batch_size=32)

# Make predictions
predictions = model.predict(X_test)

predictions = predictions.reshape(-1, 1)  # Reshape to (2520, 1) if predictions is (2520,)

# Inverse transform predictions to original scale
predictions = scaler.inverse_transform(predictions) 

# Evaluate the model
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')




D:\New folder\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 10, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,651 (119.73 KB)

 Trainable params: 30,651 (119.73 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 0.0513 - val_loss: 0.0064
Epoch 2/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0058 - val_loss: 0.0060
Epoch 3/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0058 - val_loss: 0.0058
Epoch 4/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0055 - val_loss: 0.0056
Epoch 5/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0054 - val_loss: 0.0054
Epoch 6/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0053 - val_loss: 0.0047
Epoch 7/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0048 - val_loss: 0.0044
Epoch 8/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0045 - val_loss: 0.0039
Epoch 9/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0045 - val_loss: 0.0038
Epoch 10/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0044 - val_loss: 0.0037
Epoch 11/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0044 - val_loss: 0.0038
Epoch 12/20
251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

In [10]:
# Example of predicting on new data
new_data = pd.read_csv('clearskypred.csv')  # Replace 'new1data.csv' with your new data file
new_features = new_data[['clearsky']]  # Example: selecting relevant features

# Normalize the new data using the same scaler
scaled_new_features = scaler.transform(new_features)

# Create sequences for the new data
def create_sequences(data, seq_length):
    X = []
    for i in range(len(data) - seq_length + 1):
        sequence = data[i:i + seq_length]
        X.append(sequence)
    return np.array(X)

# Example usage:
seq_length = 10  # Adjust this as per your needs
X_new = create_sequences(scaled_new_features, seq_length)

# Make predictions on new data
new_predictions = model.predict(X_new)

# Inverse transform predictions to original scale
new_predictions = scaler.inverse_transform(new_predictions)
new_predictions = new_predictions.squeeze()
# Print or use new_predictions as needed
print(f'the next 60 days  value generated {new_predictions[:60]}')
new_predictions = np.array(new_predictions)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
the next 60 days  value generated [7.1953197 7.071192  6.7272873 6.5571413 6.6258097 7.030036  7.186567
 7.200803  7.1740084 7.0381765 6.9662356 6.965393  6.918672  6.807345
 6.9489617 6.944676  7.0216517 7.0912876 7.1363473 7.115269  7.059113
 6.7496767 6.602394  6.823009  6.7314305 6.764536  6.733331  6.850695
 7.094686  7.1672783 7.201163  7.246761  7.3205476 7.2844934 7.316297
 7.319289  7.3635387 7.35688   7.459536  7.2459087 7.174908  7.054977
 6.8349915 6.9064636 6.9838095 7.0407143 7.016529  6.9456086 6.985174
 6.9816484 7.101044  7.0260987 7.04435   7.065727  7.04224   6.8588247
 6.9016495 6.969086  6.9666605 7.102627 ]


In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

class LSTMModelPredictor:
    def __init__(self, train_data_path, new_data_path):
        self.train_data_path = train_data_path
        self.new_data_path = new_data_path
        self.model = None
        self.scaler = MinMaxScaler()
        self.is_scaler_fitted = False

    def load_data(self, path):
        data = pd.read_csv(path)
        return data

    def preprocess_data(self, data):
        features = data[['clearsky']]
        target = data['temp']
        scaled_features = self.scaler.fit_transform(features)
        return scaled_features, target

    def create_sequences(self, data, seq_length):
        sequences = []
        targets = []
        for i in range(len(data) - seq_length):
            sequences.append(data[i:i + seq_length])
            targets.append(data[i + seq_length])
        return np.array(sequences), np.array(targets)

    def build_model(self, seq_length):
        self.model = Sequential()
        self.model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, 1)))
        self.model.add(LSTM(50))
        self.model.add(Dense(1))
        self.model.compile(optimizer='adam', loss='mean_squared_error')

    def train_model(self, X_train, y_train, epochs=20, batch_size=32, validation_split=0.2):
        history = self.model.fit(X_train, y_train, epochs=epochs, validation_split=validation_split, batch_size=batch_size)
        return history

    def evaluate_model(self, y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        return mse

    def predict_new_data(self, new_data, seq_length):
        scaled_new_features = self.scaler.transform(new_data)
        X_new = self.create_sequences(scaled_new_features, seq_length)
        new_predictions = self.model.predict(X_new)
        new_predictions = self.scaler.inverse_transform(new_predictions)
        new_predictions = new_predictions.squeeze()
        return new_predictions

    def run(self, seq_length=10, epochs=20, batch_size=32, validation_split=0.2):
      
          # Load training data
        train_data = self.load_data(self.train_data_path)
        # Preprocess training data
        scaled_features, target = self.preprocess_data(train_data)
        # Create sequences for LSTM
        X_train, y_train = self.create_sequences(scaled_features, seq_length)
        # Build LSTM model
        self.build_model(seq_length)
        # Train LSTM model
        self.train_model(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
        # Example of predicting on new data
        new_data = self.load_data(self.new_data_path)
        new_features = new_data[['clearsky']]
        # Predict on new data
        new_predictions = self.predict_new_data(new_features, seq_length)
        print(f'The next 60 days values predicted: {new_predictions[:60]}')
        # Return or do further processing with new_predictions if needed

# Example usage:
if __name__ == "__main__":
    # Paths to your datasets
    train_data_path = 'bangaluru data.csv'
    new_data_path = 'clearskypred.csv'
    
    # Initialize and run the LSTMModelPredictor
    lstm_predictor = LSTMModelPredictor(train_data_path, new_data_path)
    lstm_predictor.run(seq_length=10, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20


D:\New folder\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


314/314 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0541 - val_loss: 0.0066
Epoch 2/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0060 - val_loss: 0.0062
Epoch 3/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0058 - val_loss: 0.0058
Epoch 4/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0056 - val_loss: 0.0058
Epoch 5/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0055 - val_loss: 0.0046
Epoch 6/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0044 - val_loss: 0.0041
Epoch 7/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0043 - val_loss: 0.0039
Epoch 8/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0043 - val_loss: 0.0039
Epoch 9/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0044 - val_loss: 0.0038
Epoch 10/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0045 - val_loss: 0.0038
Epoch 11/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0043 - val_loss: 0.0038
Epoch 12/20
314/314 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.